In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [5]:
train=pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
train.shape

(1460, 81)

In [6]:
test=pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
test.shape

(1459, 80)

In [7]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [8]:
# using only continuos variables

train = train.select_dtypes(exclude=['object'])
train.shape

(1460, 38)

In [9]:
#filling missing values
train.drop('Id', axis=1, inplace=True)
train.fillna(0, inplace=True)

In [10]:
# test data

test=test.select_dtypes(exclude=['object'])
test.shape

ID=test['Id']

test.drop('Id', axis=1, inplace=True)
test.fillna(0, inplace=True)

In [11]:
ID

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [12]:
from sklearn.ensemble import IsolationForest

In [15]:
clf=IsolationForest(100, random_state=42)
clf.fit(train)
y_noano=clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

array([   0,    1,    2, ..., 1456, 1458, 1459])

In [16]:
train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

Number of Outliers: 84
Number of rows without outliers: 1376


In [17]:
train.head()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000


In [18]:
col_train = list(train.columns)
col_train_bis = list(train.columns)


In [19]:
col_train_bis.remove('SalePrice')

In [20]:
mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_new = np.matrix(train.drop('SalePrice',axis = 1))
mat_y = np.array(train.SalePrice).reshape((1376,1))

In [21]:
prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

MinMaxScaler()

In [22]:
prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0.235294,0.207668,0.062802,0.625,0.428571,0.948148,0.883333,0.12250,0.375532,0.0,...,0.000000,0.167582,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.332244
1,0.000000,0.255591,0.072904,0.500,0.857143,0.748148,0.433333,0.00000,0.520213,0.0,...,0.404891,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.280448
2,0.235294,0.217252,0.087396,0.625,0.428571,0.933333,0.866667,0.10125,0.258511,0.0,...,0.000000,0.115385,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.361020
3,0.294118,0.191693,0.072464,0.625,0.428571,0.296296,0.333333,0.00000,0.114894,0.0,...,0.000000,0.096154,0.824242,0.0,0.0,0.0,0.0,0.090909,0.00,0.200835
4,0.235294,0.268371,0.113835,0.750,0.428571,0.925926,0.833333,0.21875,0.348404,0.0,...,0.260870,0.230769,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.411858


In [23]:
# List of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "SalePrice"

# Columns
feature_cols = FEATURES

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS]
prediction_set = train.SalePrice

# Train and Test 
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.33, random_state=42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
training_set.head()

# Training for submission
training_sub = training_set[col_train]

In [24]:
# Same thing but for the test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
503,0.235294,0.236422,0.068125,0.750,0.428571,0.955556,0.900000,0.000000,0.000000,0.0,...,0.135870,0.096154,0.000000,0.0,0.0,0.0,0.0,0.818182,0.00,0.273733
67,0.176471,0.258786,0.125543,0.625,0.285714,0.577778,0.050000,0.000000,0.320745,0.0,...,0.000000,0.057692,0.345455,0.0,0.0,0.0,0.0,0.545455,0.00,0.363898
842,0.411765,0.204473,0.050224,0.375,0.428571,0.770370,0.483333,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.454545,0.75,0.160276
643,0.000000,0.287540,0.087379,0.875,0.428571,0.940741,0.866667,0.134375,0.563298,0.0,...,0.349185,0.123626,0.000000,0.0,0.0,0.0,0.0,0.545455,0.25,0.479001
240,0.235294,0.210863,0.065332,0.625,0.428571,0.918519,0.816667,0.188750,0.000000,0.0,...,0.260870,0.000000,0.000000,0.0,0.0,0.0,0.0,0.090909,0.00,0.373490


In [26]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

seed = 7
np.random.seed(seed)

# Model
model = Sequential()
model.add(Dense(200, input_dim=36, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
# Compile model
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())

feature_cols = training_set[FEATURES]
labels = training_set[LABEL].values

model.fit(np.array(feature_cols), np.array(labels), epochs=100, batch_size=10)

Epoch 1/100
93/93 [==============================] - 1s 1ms/step - loss: 0.0938
Epoch 2/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0917
Epoch 3/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0931
Epoch 4/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0951
Epoch 5/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0936
Epoch 6/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0942
Epoch 7/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0921
Epoch 8/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0945
Epoch 9/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0873
Epoch 10/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0961
Epoch 11/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0940
Epoch 12/100
93/93 [==============================] - 0s 1ms/step - loss: 0.0879
Epoch 13/100
93/93 [=================

In [27]:
# Evaluation on the test set created by train_test_split
model.evaluate(np.array(feature_cols), np.array(labels))


29/29 [==============================] - 0s 992us/step - loss: 0.0602


0.06024722009897232

In [30]:
# Predictions
import itertools
feature_cols_test = testing_set[FEATURES]
labels_test = testing_set[LABEL].values

y = model.predict(np.array(feature_cols_test))
predictions = list(itertools.islice(y, testing_set.shape[0]))

In [32]:
predictions = prepro_y.inverse_transform(np.array(predictions).reshape(455,1))

In [33]:
reality = pd.DataFrame(prepro.inverse_transform(testing_set), columns = [COLUMNS]).SalePrice

In [34]:
y_predict = model.predict(np.array(test))

def to_submit(pred_y,name_out):
    y_predict = list(itertools.islice(pred_y, test.shape[0]))
    y_predict = pd.DataFrame(prepro_y.inverse_transform(np.array(y_predict).reshape(len(y_predict),1)), columns = ['SalePrice'])
    y_predict = y_predict.join(ID)
    y_predict.to_csv(name_out + '.csv',index=False)
    
to_submit(y_predict, "submission_continuous")